In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/preprocessed_smiles.csv",header=None)

In [3]:
del df[1][0]

'1'

In [4]:
text = "".join(df[1][:10000])
print('corpus length:', len(text))
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))# cutting the SMILES corpus into the sequences used for training
maxlen = 40
step = 30
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
   sentences.append(text[i: i + maxlen])
   next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
   for t, char in enumerate(sentence):
       X[i, t, char_indices[char]] = 1
   y[i, char_indices[next_chars[i]]] = 1

corpus length: 728039
total chars: 36
nb sequences: 24267
Vectorization...


In [5]:
X.shape

(24267, 40, 36)

In [6]:
y.shape

(24267, 36)

In [9]:
import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [16]:
model = Sequential()
model.add(LSTM(256, input_shape=(40, 36),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(256,return_sequences=False))
model.add(Dense(36))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
                  metrics=['accuracy'])


In [17]:
model.fit(X,y,epochs=5,callbacks=[keras.callbacks.EarlyStopping(monitor="acc")])

Epoch 1/5
24267/24267 [==============================] - 252s 10ms/step - loss: 0.0854 - acc: 0.9724
Epoch 2/5
24267/24267 [==============================] - 260s 11ms/step - loss: 0.0632 - acc: 0.9774
Epoch 3/5
24267/24267 [==============================] - 252s 10ms/step - loss: 0.0571 - acc: 0.9791
Epoch 4/5
24267/24267 [==============================] - 245s 10ms/step - loss: 0.0533 - acc: 0.9805
Epoch 5/5
24267/24267 [==============================] - 249s 10ms/step - loss: 0.0508 - acc: 0.9814


In [22]:
text = "".join(df[1][10000:14000])
print('corpus length:', len(text))
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))# cutting the SMILES corpus into the sequences used for training
maxlen = 40
step = 30
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
   sentences.append(text[i: i + maxlen])
   next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
   for t, char in enumerate(sentence):
       X[i, t, char_indices[char]] = 1
   y[i, char_indices[next_chars[i]]] = 1

corpus length: 177789
total chars: 36
nb sequences: 5925
Vectorization...


In [23]:
model.evaluate(X,y)

5925/5925 [==============================] - 17s 3ms/step


[0.14200106245807453, 0.9715517988889026]